In [5]:
#tsec,osec,hashbytes = 127,128,16
#tsec,osec,hashbytes = 192,192,24
tsec,osec,hashbytes = 255,256,32

maxsigs=2**64
F = RealField(osec+100)


def pow(p,e):
    return F(p)**e
def qhitprob(qs,r):
    p = F(1/leaves)
    return binomial(qs,r)*(pow(p,r))*(pow(1-p,qs-r))

def run(l,w):
    s=l*(w-1)//2
    dp=[]
    for i in range(0,l+1):
        dp.append([])
        for j in range(0,s+1):
            dp[i].append(0)

    dp[0][0]=1
    for i in range(1,l+1):
        for j in range(0,s+1):
            for k in range(0,w):
                if k>j:
                    continue
                dp[i][j]+=dp[i-1][j-k] 
    return dp[l][s]

lenmap={}

for w in range(8,50):
    lenmap[w]=99999 
    for i in range(35,200): 
        if F(log(F(run(i,w)))/log2)>osec:
            lenmap[w]=i 
            print(w,i)
            break
print('preprocess end')

8 88
9 83
10 79
11 76
12 74
13 71
14 69
15 68
16 66
17 65
18 63
19 62
20 61
21 60
22 59
23 59
24 58
25 57
26 56
27 56
28 55
29 55
30 54
31 54
32 53
33 53
34 52
35 52
36 51
37 51
38 51
39 50
40 50
41 50
42 49
43 49
44 49
45 49
46 48
47 48
48 48
49 48
preprocess end


In [7]:
max_size=49856
max_hashcalls=327696

def size(h,d,b,k,w1,w2):
    l=lenmap[w1]
    return ((b+1)*k+h+l*d+1)*hashbytes


def split(h,d):
    ans=[0 for i in range(d)]
    for i in range(h):
        ans[i%d]+=1
    return ans

def speed(h,d,b,k,w1,w2):
    l=lenmap[w1]
    ans=k*(2**b)*(2+w2-1)
    ans+=d*(2**(h/d)*(l*w1+1))
    return ans


def sec(h,d,b,k,w1,w2):
    s=F(0)
    for r in range(100): # in fact, qhitprob(maxsigs,r)<2^{-256}*2^{-64} when r>100. Thus we can omit all r>100
        p=F(1-(1-1/F(2**b)*F((w2+1)/(2*w2)))^r)**k
        s+=qhitprob(maxsigs,r)*p
    return -F(log(F(s))/log2)

def factor(h):
    ans=[]
    for d in range(6,h):
        if h%d==0:
            ans.append(d)
    return ans

for h in [63,64,65,66]:
    leaves=2**h
    for w2 in [1,2,4,8,16]:
        for b in range(3,15):
            for k in range(8,100):
                if sec(h,0,b,k,0,w2)>tsec:
                    for d in factor(h):
                        for w1 in range(8,50): 
                            if sec(h,d,b,k,w1,w2)>tsec and size(h,d,b,k,w1,w2)<=max_size and speed(h,d,b,k,w1,w2)<max_hashcalls:
                                print(h,d,b,k,w1,w2)
                                print(size(h,d,b,k,w1,w2))
                                print(speed(h,d,b,k,w1,w2))
                    break


64 16 8 53 18 1
49600
317696
64 16 9 43 16 1
49632
314624
64 16 9 43 17 1
49120
327168
64 16 7 62 19 2
49696
325632
64 16 8 48 16 2
49696
307456
64 16 8 48 17 2
49184
320000
64 16 8 48 18 2
48160
327424
64 16 9 40 15 2
49696
322816
64 16 7 58 18 4
49184
327680
64 16 8 45 15 4
49856
318976
